In [29]:
import time
from time import sleep
import re
from tqdm import tqdm
from random import randint

# Import BeautifulSoup and requests module
import requests
from bs4 import BeautifulSoup as bs

In [13]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; InteSl Mac O X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
my_url = "http://www.pythonscraping.com/exercises/exercise1.html"

# 사이트 요청 
response = requests.get(my_url, headers=headers)

# 요청의 status code 보기 
response.status_code

200

In [14]:
# HTML 보기
response.text

'<html>\n<head>\n<title>A Useful Page</title>\n</head>\n<body>\n<h1>An Interesting Title</h1>\n<div>\nLorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n</div>\n</body>\n</html>\n'

# BeautifulSoup 

### Find & FindAll

In [44]:
# Title tag 찾기 
soup = bs(response.text)
soup.find('title')

In [ ]:
# Title tag 내용 


#### CSS Selector

In [22]:
url = "http://www.pythonscraping.com/pages/warandpeace.html"
response = requests.get(url, headers=headers)
response.status_code

200

In [34]:
# 초록색 단어 찾기
soup = bs(response.text, 'html.parser')
soup.find('span', {"class" : "green"}).text

'Anna\nPavlovna Scherer'

In [36]:
# 모든 초록색 단어 찾기 (findAll)
soup = bs(response.text, 'html.parser')
soup.findAll('span', {"class" : "green"})

green_word_tags = soup.findAll('span',class_="green")

SyntaxError: keyword can't be an expression (<ipython-input-36-bfbbbb60fc37>, line 4)

In [42]:
# 빨간색 단어를 추출해봅시다 
red_word_tags = soup.findAll('span',class_="red")

In [47]:
# <span> 태그에 존재하는 빨간색과 초록색 글씨를 모두 추출해봅시다
soup.findAll('span', {"class" : "red", "class" : "green"})

AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [48]:
# 복습: <p> 에 있는 내용을 모두 리스트에 저장해봅시다. 
paragraph_tags = soup.findAll('p')

### 실제 사이트에 연습:

In [58]:
url = 'http://www.daum.net'

#Request 요청 (headers 추가 필수)
response = requests.get(url, headers=headers)
soup = bs(response.text)

# 탭버튼이 담긴 정보 찾기 
big_tab = soup.find('ul',{"class":"list_mainsvc"})

#findAll 도 해보기
for i in big_tab.findAll('a'):
    print(i.text)

카페
메일
뉴스
지도
증권
쇼핑
카카오TV
웹툰
블로그
브런치
사전
게임
같이가치


#### 태그 안에 있는 URL 정보를 얻고 싶다면?

In [60]:
# Find 문 이용시 
tabs = big_tab.findAll('a')
cafe = tabs[0]
cafe['href']
# FindALL 문 이용시 


'http://cafe.daum.net/'

### Parent & Sibling 관계 이용하기

In [71]:
url = "http://pythonscraping.com/pages/page3.html"
response = requests.get(url)
soup = bs(response.text)

In [75]:
# 첫번째 이미지 찾기 
first_image_table = soup.find('img',{'src': "../img/gifts/img1.jpg"})

In [77]:
# parent& sibling 관계를 이용하여
# 다음 행에 있는 이미지 URL 구하기! 
first_image_table.parent.previous_sibling.text

'\n$15.00\n'

In [78]:
# 복습: findAll 문을 이용하여 모든 이미지 URl 구하기
images_tags = soup.findAll('img')
for i in images_tags:
    print(i['src'])

../img/gifts/logo.jpg
../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


In [ ]:
# 복습: CSS 문을 이용하여 볼드체 글씨를 모두 구하기

### Regular Expression 이용하기

In [79]:
import re
# 정규식을 활용하여 gift 이미지 URL만 구하기 
# 정규식은 제공 
gift_regex = re.compile('img\d.jpg')
soup.findAll('img',{"src":gift_regex})

[<img src="../img/gifts/img1.jpg"/>,
 <img src="../img/gifts/img2.jpg"/>,
 <img src="../img/gifts/img3.jpg"/>,
 <img src="../img/gifts/img4.jpg"/>,
 <img src="../img/gifts/img6.jpg"/>]

# 실제 크롤링 연습:  다음 영화 

In [91]:
def daum_movie_crawler(year):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; InteSl Mac O X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    main_url = 'https://movie.daum.net/boxoffice/yearly?year=' + str(year)
    
    # Initialize 
    data = []
    cnt= 0 
    
    response = requests.get(main_url,headers=headers)
    body_table = bs(response.text,'html.parser')
    
    #포스터 테이블 구하기 
    poster_table = body_table.findAll('div',{"class" : 'wrap_movie'})

    
    #포스터 테이블을 looping 
    # 제목과 평론가 평점 찾기
    for poster in poster_table:
        title_tag = poster.find('a',{'class' : "name_movie #title"})
        data.append([title_tag.text])                                 
                                                      
    return data

daum_movie_crawler(2014)

[['명량'],
 ['겨울왕국'],
 ['인터스텔라'],
 ['해적: 바다로 간 산적'],
 ['수상한 그녀'],
 ['변호인'],
 ['국제시장'],
 ['트랜스포머: 사라진 시대'],
 ['군도: 민란의 시대'],
 ['엣지 오브 투모로우'],
 ['엑스맨: 데이즈 오브 퓨처 패스트'],
 ['어메이징 스파이더맨 2'],
 ['타짜-신의 손'],
 ['혹성탈출: 반격의 서막'],
 ['캡틴 아메리카: 윈터 솔져'],
 ['역린'],
 ['님아, 그 강을 건너지 마오'],
 ['신의 한 수'],
 ['끝까지 간다'],
 ['비긴 어게인'],
 ['드래곤 길들이기 2'],
 ['표적'],
 ['메이즈 러너'],
 ['호빗: 다섯 군대 전투'],
 ['나의 사랑 나의 신부'],
 ['논스톱'],
 ['인투 더 스톰'],
 ['노아'],
 ['남자가 사랑할 때'],
 ['루시'],
 ['용의자'],
 ['기술자들'],
 ['나를 찾아줘'],
 ['제보자'],
 ['피끓는 청춘'],
 ['두근두근 내 인생'],
 ['우아한 거짓말'],
 ['300: 제국의 부활'],
 ['엑소더스: 신들과 왕들'],
 ['해무(海霧)'],
 ['인간중독'],
 ['말레피센트'],
 ['폼페이: 최후의 날'],
 ['퓨리'],
 ['드라큘라: 전설의 시작'],
 ['가디언즈 오브 갤럭시'],
 ['찌라시: 위험한 소문'],
 ['빅매치'],
 ['슬로우 비디오'],
 ['방황하는 칼날']]

In [ ]:
#2009-2019년도 데이터 얻기 

# 예외/ 오류 처리

In [ ]:
try: 
    response = requests.get(my_url, headers=headers)
except Exception as e:
    print('Error occured while requesting')

if response.status_code == 200:
    soup = bs(response.text, 'html.parser')
else:
    print('Status code Error ')
    

In [ ]:
"""
위에 daum_movie_crawler 에 try-except 문을 추가해서
오류가 나는 것을 방지해보기 
"""

# Selenium 

In [103]:
import selenium 
from selenium import webdriver
import random
from time import sleep

browser = webdriver.Chrome()

In [104]:
your_url = "https://www.ncbi.nlm.nih.gov/pubmed/?term=wine"
browser = webdriver.Chrome() #혹은 .Firefox() , .Safari()
browser.get(your_url)

In [105]:
# 다음으로 가기 버튼 xpath 를 이용해 찾기 
next_button = browser.find_element_by_xpath('//*[@id="EntrezSystem2.PEntrez.PubMed.Pubmed_ResultsPanel.Pubmed_Pager.Page"]')

# 버튼 클릭 
next_button.click()

In [107]:
#논문 태그 찾기 (class 이용)
paper_tags = browser.find_elements_by_class_name('rprt')


In [110]:
# 각 태그의 제목 (tag_name 이용)
for paper in paper_tags : 
    print(paper.find_element_by_tag_name('a').text)
    

The Combination of Hot Air and Chitosan Treatments on Phytochemical Changes during Postharvest Storage of 'Sanhua' Plum Fruits.
Influence of fungicide residues and in vitro gastrointestinal digestion on total antioxidant capacity and phenolic fraction of Graciano and Tempranillo red wines.
Comparative transcriptional analysis of flavour-biosynthetic genes of a native Saccharomyces cerevisiae strain fermenting in its natural must environment, vs. a commercial strain and correlation of the genes' activities with the produced flavour compounds.
Targeted Drug Delivery in Plants: Enzyme-Responsive Lignin Nanocarriers for the Curative Treatment of the Worldwide Grapevine Trunk Disease Esca.
Potential of Cooperage Byproducts Rich in Ellagitannins to Improve the Antioxidant Activity and Color Expression of Red Wine Anthocyanins.
Pectolytic enzyme reduces the concentration of colloidal particles in wine due to changes in polysaccharide structure and aggregation properties.
Indigenous Yeast Inte

In [ ]:
"""
5 페이지동안 논문제목을 크롤링한뒤 
논문제목이 담긴 리스트를 반환하는 함수를 구현해보세요 (다음 페이지로 넘어가기전에 0-1초간 휴식)
"""


browser = webdriver.Chrome() 
browser.get(your_url)

for page in range(1,6):
    print("Currently on page ", page)
    paper_tags = browser.find_elements_by_class_name('rprt')
    for paper in paper_tags : 
        #paper 찾기
        #data.append(paper 제목)
    #next_button 찾기
    #next_buttn 클릭하기
    sleep(random.uniform(0,1))
    